In [1]:
import pandas as pd
import dtale
import numpy as np

In [3]:
df = pd.DataFrame()
chunksize = 10 ** 3

# read the dataset by splitting it by chunks because this dataset is too large (13GB), it leads to kernel crashes
with pd.read_csv('../../datasets/CICIOT2023/merged_CICIOT2023.csv', chunksize=chunksize) as reader:
    for chunk in reader:
        df = pd.concat([df, chunk], ignore_index=True)
        progress = (len(df) / 13754096325) * 100
        print(f"\rReading CSV: {progress:.2f}% complete", end="")

#print(df)

Reading CSV: 0.02% complete

KeyboardInterrupt: 

### Missing datas
useless here because no missing datas

In [ ]:
from sklearn.impute import SimpleImputer # to handle missing data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("Missing datas by columns :\n", df.isnull().sum())

#here there is no missing data so we don't have to manage this

df.replace([np.inf, -np.inf], np.nan, inplace=True)
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

Missing datas by columns :
  Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64


### Separing datas and labels

In [ ]:
df.columns = df.columns.str.strip() # to clean spaces on the columns names
X = df.drop(columns=['Label']) # without labels
Y = df['Label'] # just labels

### Encoding categorical data
Difference between Label and OneHot : \
                                      - Label gives a number in int for each line (simple but the model could misunderstand the difference between numbers (priorities issues))\
                                      - OneHot encodes in binary columns (no hierarchy between column but a lot of columns if they are a lot of categories to label)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# no need of OneHotEncoder because all columns except Label are numerical columns

Y = Y.apply(lambda x: 0 if x == 'BENIGN' else 1) # because we have several types of attacks and we wants bianaries Y

print(Y)

0          0
1          0
2          0
3          0
4          0
          ..
2830738    0
2830739    0
2830740    0
2830741    0
2830742    0
Name: Label, Length: 2830743, dtype: int64


### Splitting into training set and test set
We are now splitting the dataset
The train set that has the full data to train and the test set which has only 3 columns for testing on smaller samples

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
print(X_train)

         Destination Port  Flow Duration  Total Fwd Packets  \
2092860              53.0        64428.0                2.0   
2140044             443.0       757414.0               12.0   
1590488              53.0       184165.0                4.0   
128477            54342.0      1863306.0                6.0   
1336329              53.0          235.0                2.0   
...                   ...            ...                ...   
2249467              80.0     85562173.0                8.0   
963395              443.0    118654701.0               45.0   
2215104              80.0     87497358.0                8.0   
1484405             443.0      1543512.0                8.0   
305711               80.0    115280042.0               16.0   

         Total Backward Packets  Total Length of Fwd Packets  \
2092860                     2.0                         78.0   
2140044                     8.0                        545.0   
1590488                     2.0                    

### Feature Scaling
To have the same scale between each columns (for instance, `Flow Duration` is way bigger than `Tot Fwd Pkts` and the model could misinterpret it and gives more importance to the Income column)